In [1]:
import numpy as np
import pandas as pd
import datetime
from airflow.providers.postgres.hooks.postgres import PostgresHook

warehouse_hook = PostgresHook(postgres_conn_id='rds_afsg_ds_prod_postgresql_dwh', schema='afsg_ds_prod_postgresql_dwh')

/tmp/ipykernel_13349/4044894059.py:6 DeprecationWarning: The "schema" arg has been renamed to "database" as it contained the database name.Please use "database" to set the database name.

#### Files

# Safaricom Bloom

### Digital Loans Account Data

In [4]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('SB2_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('SB2_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        case when cmsv.fuzzy_match_score >= 60
            then concat(cmsv.iprs_first_name, ' ', cmsv.iprs_other_name, ' ', cmsv.iprs_surname)
            else concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name)
        end as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from bloomlive.loans_fact_table_materialized_summary_view lftmsv
    inner join bloomlive.client_materialized_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join bloomlive.product_dimension pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join bloomlive.loan_status_dimension lsd on lftmsv.loan_status_key = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where lftmsv.bloom_version = '2' and loan_status in (300, 600, 700)
    limit 100
    """
).to_csv('~/data/Digital Loans Account Data Sample Bloom 2.0.csv', index=False, sep='|')

[2023-07-07 11:14:38,663] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

### Overdue (Non-Performing) Digital Loans

In [5]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('SB2_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('SB2_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from bloomlive.loans_fact_table_materialized_summary_view lftmsv
    inner join bloomlive.client_materialized_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300 and lftmsv.bloom_version = '2'
    limit 100
    """
).to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample Bloom 2.0.csv', index=False, sep='|')

[2023-07-07 11:19:24,462] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

### Digital Loan Repayments Data

In [7]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    SELECT row_number() OVER () AS "ROW_ID",
    '0900024'::text AS "DIGITAL_CREDIT_PROVIDER_CODE",
    to_char(CURRENT_DATE::timestamp with time zone, 'DD-Mon-YYYY'::text) AS "REPORTING_DATE",
    concat('SB2_', cmsv.mifos_id) AS "CUSTOMER_ID",
    concat('SB2_', lftmsv.loan_mifos_id) AS "LOAN_ACCOUNT_CONTRACT_NUMBER",
    COALESCE(lftmsv.total_outstanding, 0::numeric) AS "OUTSTANDING_LOAN_BALANCE",
    COALESCE(lftmsv.interest_outstanding, 0::numeric) AS "ACCRUED_INTEREST_BALANCE",
    0 AS "INTEREST_CHARGED_ON_REPAYMENT",
    COALESCE(lftmsv.fee_charges_charged + lftmsv.penalty_charges_charged, 0::numeric) AS "LATE_PAYMENT_FEE_CHARGE",
    to_char(td.submitted_on_date::timestamp with time zone, 'DD-Mon-YYYY'::text) AS "LOAN_REPAYMENT_DATE",
    COALESCE(lftmsv.principal_repaid, 0::numeric) AS "PRINCIPAL_LOAN_AMOUNT_REPAID"
     from bloomlive.loans_fact_table_materialized_summary_view lftmsv
     inner JOIN bloomlive.client_materialized_summary_view cmsv ON lftmsv.client_surrogate_id = cmsv.surrogate_id
     inner join (
        with rnked as (
            select submitted_on_date, mifos_loan_id, bloom_version, rank() over (partition by mifos_loan_id, bloom_version order by transaction_date desc, transaction_id desc) rnk
            from bloomlive.transactions_dimension td
            where td.transaction_type_enum = 2 AND td.is_reversed::integer = 0 AND (lower(td.receipt_number::text) !~~ '%\_r%'::text AND lower(td.receipt_number::text) !~~ '%waiv%'::text OR td.receipt_number IS NULL)
        ) select mifos_loan_id, bloom_version, submitted_on_date from rnked where rnk = 1
     ) td ON lftmsv.loan_mifos_id::character varying::text = td.mifos_loan_id::text AND lftmsv.bloom_version::text = td.bloom_version::text
  WHERE lftmsv.bloom_version = '2' and lftmsv.loan_status = 300
    limit 100
    """
).to_csv('~/data/Digital Loan Repayments Data Sample Bloom 2.0.csv', index=False, sep='|')

[2023-07-07 12:06:28,951] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


### Customer Data

In [3]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('SB2_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.idm_gender,
            case when cmsv.fuzzy_match_score >= 60 then cmsv.iprs_first_name else cmsv.first_name end as "FIRST_NAME",
            case when cmsv.fuzzy_match_score >= 60 then cmsv.iprs_other_name else cmsv.middle_name end as "MIDDLE_NAME",
            case when cmsv.fuzzy_match_score >= 60 then cmsv.iprs_surname else cmsv.last_name end as "SURNAME",
            to_char(cmsv.idm_date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from bloomlive.client_materialized_summary_view cmsv
        inner join bloomlive.loans_fact_table_materialized_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
        where cmsv.bloom_version = '2' and idm_gender is not null and idm_date_of_birth is not null
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.idm_gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
).to_csv(f'~/data/CBK Customer Data Sample Bloom 2.0 {datetime.datetime.now()}.csv', index=False, sep='|')

[2023-07-07 10:41:32,100] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

# SOLV

### Digital Loans Account Data

In [8]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('S_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('S_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from solv_bat.loans_fact_table_summary_view lftmsv
    inner join solv_bat.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join solv_bat.product_dimension pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join solv_bat.loan_status_dimension lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample SOLV.csv', index=False)

[2023-06-29 10:14:16,652] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,29-Jun-2023,S2_8330,S2_6595,EVANS GICHANA OGAKE,17-Jun-2022,50000.0,0.0,0,750.0,0,0,0,200.0,CHN02,7,50950.0,CLOSED
1,2,0900024,29-Jun-2023,S2_8330,S2_6727,EVANS GICHANA OGAKE,24-Jun-2022,100000.0,0.0,0,1500.0,0,0,0,1726.0,CHN02,7,103226.0,CLOSED
2,3,0900024,29-Jun-2023,S2_8330,S2_7504,EVANS GICHANA OGAKE,29-Jul-2022,50000.0,0.0,0,901.0,0,0,0,200.0,CHN02,7,51101.0,CLOSED
3,4,0900024,29-Jun-2023,S2_8330,S2_7635,EVANS GICHANA OGAKE,04-Aug-2022,100000.0,0.0,0,1801.0,0,0,0,200.0,CHN02,7,102001.0,CLOSED
4,5,0900024,29-Jun-2023,S2_8330,S2_8078,EVANS GICHANA OGAKE,26-Aug-2022,100000.0,0.0,0,1801.0,0,0,0,200.0,CHN02,7,102001.0,CLOSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,29-Jun-2023,S2_9426,S2_8359,Fredrick Loole,08-Sep-2022,32092.0,0.0,0,1156.0,0,0,0,1481.0,CHN02,14,34729.0,CLOSED
96,97,0900024,29-Jun-2023,S2_9453,S2_8425,Lilian Nangulu,10-Sep-2022,40760.0,0.0,0,1468.0,0,0,0,200.0,CHN02,14,42428.0,CLOSED
97,98,0900024,29-Jun-2023,S2_9453,S2_8785,Lilian Nangulu,24-Sep-2022,43200.0,0.0,0,1556.0,0,0,0,200.0,CHN02,14,44956.0,CLOSED
98,99,0900024,29-Jun-2023,S2_9454,S2_8426,Lucy Wakonyo Wamaitha,10-Sep-2022,37400.0,0.0,0,1347.0,0,0,0,200.0,CHN02,14,38947.0,CLOSED


### Overdue (Non-Performing) Digital Loans

In [11]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('S_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('S_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from solv_bat.loans_fact_table_summary_view lftmsv
    inner join solv_bat.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample SOLV.csv', index=False)

[2023-06-29 10:16:58,058] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT
0,1,0900024,29-Jun-2023,S_10203,S_11260,23-May-2023,78000.0,126488.41
1,2,0900024,29-Jun-2023,S_10315,S_11269,23-May-2023,60000.0,114049.00
2,3,0900024,29-Jun-2023,S_9480,S_11285,18-May-2023,4949.0,4949.00
3,4,0900024,29-Jun-2023,S_9448,S_11344,24-May-2023,8899.0,8899.00
4,5,0900024,29-Jun-2023,S_10395,S_11349,24-May-2023,24779.0,24779.00
...,...,...,...,...,...,...,...,...
95,96,0900024,29-Jun-2023,S_10222,S_10374,02-Apr-2023,57400.0,77681.47
96,97,0900024,29-Jun-2023,S_9511,S_8536,28-Dec-2022,99996.0,157945.97
97,98,0900024,29-Jun-2023,S_9607,S_8743,04-Jan-2023,35930.0,53054.52
98,99,0900024,29-Jun-2023,S_10227,S_10586,11-Apr-2023,35000.0,69353.04


### Digital Loan Repayments Data

In [17]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('S_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('S_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from solv_bat.transactions_dimension td
    inner join solv_bat.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join solv_bat.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample SOLV.csv', index=False)

[2023-06-29 10:24:29,254] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,29-Jun-2023,S_10475,S_12054,0.00,0.0,0,0.0,24-Apr-2023,41000.00
1,2,0900024,29-Jun-2023,S_10420,S_12056,46883.00,0.0,0,0.0,20-Apr-2023,40000.00
2,3,0900024,29-Jun-2023,S_10420,S_12056,117727.00,0.0,0,0.0,18-Apr-2023,32273.00
3,4,0900024,29-Jun-2023,S_10420,S_12056,86883.00,0.0,0,0.0,19-Apr-2023,30000.00
4,5,0900024,29-Jun-2023,S_10420,S_12056,844.00,0.0,0,0.0,24-Apr-2023,46883.00
...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,29-Jun-2023,S_10140,S_11293,49500.00,0.0,0,0.0,20-Feb-2023,0.00
96,97,0900024,29-Jun-2023,S_10386,S_11294,0.00,0.0,0,0.0,22-Feb-2023,17856.62
97,98,0900024,29-Jun-2023,S_10386,S_11294,17856.62,0.0,0,0.0,20-Feb-2023,3843.38
98,99,0900024,29-Jun-2023,S_9369,S_11295,0.00,0.0,0,0.0,18-Feb-2023,60000.00


### Customer Data

In [19]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('S_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from solv_bat.client_summary_view cmsv
        inner join solv_bat.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-06-29 10:25:29,535] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,29-Jun-2023,SB2_9957,CUSTID01,1502421,None,KE,None,Robert,Dacho,Amollo,19-Oct-2012,19-Oct-2022,254716948361
1,2,0900024,29-Jun-2023,SB2_10047,CUSTID01,31622017,None,KE,None,Regina,None,Muthoni,27-Oct-2012,27-Oct-2022,254706177442
2,3,0900024,29-Jun-2023,SB2_8990,CUSTID01,27027781,None,KE,None,EVANS,None,SMOOTH,27-Jul-2012,27-Jul-2022,254728734720
3,4,0900024,29-Jun-2023,SB2_9439,CUSTID01,13213664,None,KE,None,FRANCIS,None,MWANGI,09-Sep-2012,09-Sep-2022,254729143588
4,5,0900024,29-Jun-2023,SB2_9702,CUSTID01,23540208,None,KE,None,Agnes,Wanjiru,Thuita,30-Sep-2012,30-Sep-2022,254713530581
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,29-Jun-2023,SB2_10012,CUSTID01,13653582,None,KE,None,Joyce,Wanjiru,Wangombe,25-Oct-2012,25-Oct-2022,254723354617
96,97,0900024,29-Jun-2023,SB2_9727,CUSTID01,20740614,None,KE,None,Caroline,None,Nkatha,03-Oct-2012,03-Oct-2022,254723908291
97,98,0900024,29-Jun-2023,SB2_10078,CUSTID01,12992655,None,KE,None,Benard,Gitau,Dorcas,03-Nov-2012,03-Nov-2022,254722575352
98,99,0900024,29-Jun-2023,SB2_10034,CUSTID01,12528132,None,KE,None,Susan,None,Kaguura,27-Oct-2012,27-Oct-2022,254726797765


# TANDA

### Digital Loans Account Data

In [3]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('T_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('T_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from tanda.loans_fact_table_summary_view lftmsv
    inner join tanda.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join tanda.product_dimension pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join tanda.loan_status_dimension lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample TANDA.csv', index=False)

[2023-07-05 09:23:34,468] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,05-Jul-2023,T_7,T_1,Geoffrey Otieno,15-Oct-2020,2000.0,0.0,0,30.00,0,0,0,0.00,CHN02,3,2030.00,CLOSED
1,2,0900024,05-Jul-2023,T_7,T_30,Geoffrey Otieno,25-Nov-2020,3800.0,0.0,0,95.00,0,0,0,0.00,CHN02,7,3895.00,CLOSED
2,3,0900024,05-Jul-2023,T_16,T_2,Lindah Mugambi,15-Oct-2020,4869.0,0.0,0,121.73,0,0,0,0.02,CHN02,7,4990.75,CLOSED
3,4,0900024,05-Jul-2023,T_19,T_3,Maureen Wairimu,16-Oct-2020,2000.0,0.0,0,50.00,0,0,0,41.00,CHN02,7,2091.00,CLOSED
4,5,0900024,05-Jul-2023,T_7,T_150,Geoffrey Otieno,12-Apr-2021,5500.0,0.0,0,247.00,0,0,0,0.00,CHN02,7,5747.00,CLOSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,T_1001,T_2476,JOSIAH MUTEMI,22-Nov-2021,10000.0,0.0,0,450.00,0,0,0,262.00,CHN02,7,10712.00,CLOSED
96,97,0900024,05-Jul-2023,T_2664,T_2477,simon wainaina,22-Nov-2021,12000.0,0.0,0,540.00,0,0,0,0.00,CHN02,7,12540.00,CLOSED
97,98,0900024,05-Jul-2023,T_738,T_2478,Gad Mangi,22-Nov-2021,3800.0,0.0,0,171.00,0,0,0,0.00,CHN02,7,3971.00,CLOSED
98,99,0900024,05-Jul-2023,T_976,T_2479,BONIFACE OTUNGA,22-Nov-2021,10000.0,0.0,0,450.00,0,0,0,0.00,CHN02,7,10450.00,CLOSED


### Overdue (Non-Performing) Digital Loans

In [12]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('T_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('T_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from tanda.loans_fact_table_summary_view lftmsv
    inner join tanda.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample TANDA.csv', index=False)

[2023-07-05 10:02:46,270] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT
0,1,0900024,05-Jul-2023,T_2310,T_1230,30-Dec-2021,10000.0,12933.0
1,2,0900024,05-Jul-2023,T_1515,T_317,16-Oct-2021,2800.0,3378.0
2,3,0900024,05-Jul-2023,T_765,T_699,25-Nov-2021,184.0,184.0
3,4,0900024,05-Jul-2023,T_3213,T_1274,02-Jan-2022,2100.0,2873.0
4,5,0900024,05-Jul-2023,T_2468,T_1299,02-Jan-2022,10000.0,10549.0
...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,T_2333,T_2945,26-Mar-2022,15000.0,20420.0
96,97,0900024,05-Jul-2023,T_1001,T_3143,08-Apr-2022,14200.0,17596.0
97,98,0900024,05-Jul-2023,T_3466,T_2888,22-Mar-2022,7679.0,7679.0
98,99,0900024,05-Jul-2023,T_3496,T_3161,10-Apr-2022,10000.0,13874.0


### Digital Loan Repayments Data

In [7]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('T_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('T_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from tanda.transactions_dimension td
    inner join tanda.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join tanda.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample TANDA.csv', index=False)

[2023-07-05 09:25:18,531] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,05-Jul-2023,T_83,T_254,1208.0,0.0,0,0.0,05-Jul-2021,8792.0
1,2,0900024,05-Jul-2023,T_3387,T_1266,11.0,0.0,0,0.0,04-Oct-2021,2500.0
2,3,0900024,05-Jul-2023,T_27,T_1432,8000.0,0.0,0,0.0,04-Oct-2021,0.0
3,4,0900024,05-Jul-2023,T_39,T_1430,442.0,0.0,0,0.0,13-Oct-2021,4137.0
4,5,0900024,05-Jul-2023,T_895,T_2173,0.0,0.0,0,0.0,16-Nov-2021,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,T_1338,T_10387,18200.0,983.0,0,0.0,02-Mar-2023,0.0
96,97,0900024,05-Jul-2023,T_4970,T_11981,2223.0,0.0,0,2323.0,16-Apr-2023,17777.0
97,98,0900024,05-Jul-2023,T_4970,T_12055,16634.0,0.0,0,0.0,17-Apr-2023,2223.0
98,99,0900024,05-Jul-2023,T_4970,T_12055,18857.0,0.0,0,0.0,17-Apr-2023,1143.0


[2023-07-05 09:25:21,915] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,05-Jul-2023,T_83,T_254,1208.0,0.0,0,0.0,05-Jul-2021,8792.0
1,2,0900024,05-Jul-2023,T_3387,T_1266,11.0,0.0,0,0.0,04-Oct-2021,2500.0
2,3,0900024,05-Jul-2023,T_27,T_1432,8000.0,0.0,0,0.0,04-Oct-2021,0.0
3,4,0900024,05-Jul-2023,T_39,T_1430,442.0,0.0,0,0.0,13-Oct-2021,4137.0
4,5,0900024,05-Jul-2023,T_895,T_2173,0.0,0.0,0,0.0,16-Nov-2021,2000.0
...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,T_1338,T_10387,18200.0,983.0,0,0.0,02-Mar-2023,0.0
96,97,0900024,05-Jul-2023,T_4970,T_11981,2223.0,0.0,0,2323.0,16-Apr-2023,17777.0
97,98,0900024,05-Jul-2023,T_4970,T_12055,16634.0,0.0,0,0.0,17-Apr-2023,2223.0
98,99,0900024,05-Jul-2023,T_4970,T_12055,18857.0,0.0,0,0.0,17-Apr-2023,1143.0


### Customer Data

In [9]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('T_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from tanda.client_summary_view cmsv
        inner join tanda.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-07-05 09:26:21,389] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,05-Jul-2023,T_3576,CUSTID01,21217542,None,KE,None,Eclai,None,Andika,10-Mar-1979,13-Sep-2021,254706134475
1,2,0900024,05-Jul-2023,T_3174,CUSTID01,37437145,None,KE,None,BELINDA,None,ANAM,04-Apr-1999,24-Aug-2021,254795302473
2,3,0900024,05-Jul-2023,T_8557,CUSTID01,38662080,None,KE,None,Nixon,None,Kipchumbq,24-Aug-2001,28-Jun-2022,254795701443
3,4,0900024,05-Jul-2023,T_40,CUSTID01,22144822,None,KE,None,john,None,mugo,08-May-1979,16-Oct-2020,254716622477
4,5,0900024,05-Jul-2023,T_6688,CUSTID01,33224974,None,KE,None,Benson,None,Kandagor,27-May-1996,29-Mar-2022,254707045240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,T_4017,CUSTID01,28466866,None,KE,None,kombo,None,zuma,31-Dec-1984,07-Oct-2021,254728326882
96,97,0900024,05-Jul-2023,T_4540,CUSTID01,39346789,None,KE,None,PHILIP,None,TABITHA,24-Jul-2000,02-Nov-2021,254758452903
97,98,0900024,05-Jul-2023,T_7179,CUSTID01,37670941,None,KE,None,Elicose,None,chemutai,26-Jun-1998,27-Apr-2022,254713958747
98,99,0900024,05-Jul-2023,T_2754,CUSTID01,25408708,None,KE,None,Davis,None,Saikwa,18-Jun-1987,06-Aug-2021,254728750803


# JUMIA

### Digital Loans Account Data

In [11]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('J_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('J_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from jumia.loans_fact_table_summary_view lftmsv
    inner join jumia.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join jumia.product_dimension pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join jumia.loan_status_dimension lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample TANDA.csv', index=False)

[2023-07-05 09:39:12,258] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,05-Jul-2023,J_5028,J_2719,Moses Mogaka,06-Dec-2021,26600.0,13542.0,0,1670.0,0,0,0,0.0,CHN02,1,28270.0,ACTIVE
1,2,0900024,05-Jul-2023,J_5392,J_4108,Victor Ochieng' Ogol,16-Feb-2022,5000.0,1.0,0,314.0,0,0,0,0.0,CHN02,1,5314.0,ACTIVE
2,3,0900024,05-Jul-2023,J_7570,J_6009,Josphine Wanjiku Kamande,17-May-2022,5900.0,8287.0,0,371.0,0,0,0,2016.0,CHN02,1,8287.0,ACTIVE
3,4,0900024,05-Jul-2023,J_8593,J_6843,Iddolyne Kerubo,30-Jun-2022,2900.0,335.0,0,183.0,0,0,0,483.0,CHN02,1,3566.0,ACTIVE
4,5,0900024,05-Jul-2023,J_9344,J_8191,Titus Kyalo,31-Aug-2022,5300.0,7459.0,0,400.0,0,0,0,40932.0,CHN02,1,46632.0,ACTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,J_5393,J_2994,Tony Obare,22-Dec-2021,2400.0,0.0,0,151.0,0,0,0,0.0,CHN02,1,2551.0,CLOSED
96,97,0900024,05-Jul-2023,J_5394,J_2995,David Mungai Wanjiku,22-Dec-2021,9800.0,0.0,0,615.0,0,0,0,5816.0,CHN02,1,16231.0,CLOSED
97,98,0900024,05-Jul-2023,J_5394,J_9534,David Mungai Wanjiku,26-Oct-2022,2300.0,0.0,0,188.0,0,0,0,0.0,CHN02,1,2488.0,CLOSED
98,99,0900024,05-Jul-2023,J_5394,J_10074,David Mungai Wanjiku,30-Nov-2022,2300.0,0.0,0,188.0,0,0,0,272.0,CHN02,1,2760.0,CLOSED


### Overdue (Non-Performing) Digital Loans

In [13]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('J_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('J_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from jumia.loans_fact_table_summary_view lftmsv
    inner join jumia.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample TANDA.csv', index=False)

[2023-07-05 10:03:26,663] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT
0,1,0900024,05-Jul-2023,T_6082,T_3999,06-Jul-2022,4700.0,6500.0
1,2,0900024,05-Jul-2023,T_7443,T_5907,04-Oct-2022,19500.0,28373.0
2,3,0900024,05-Jul-2023,T_5265,T_2797,05-May-2022,34239.0,61971.0
3,4,0900024,05-Jul-2023,T_5623,T_5121,25-Aug-2022,7800.0,10762.0
4,5,0900024,05-Jul-2023,T_7517,T_5968,06-Oct-2022,12500.0,13724.0
5,6,0900024,05-Jul-2023,T_6178,T_4130,13-Jul-2022,20000.0,25438.0
6,7,0900024,05-Jul-2023,T_6174,T_5571,16-Sep-2022,15100.0,19660.0
7,8,0900024,05-Jul-2023,T_8878,T_7255,14-Dec-2022,1442.0,1827.0
8,9,0900024,05-Jul-2023,T_7570,T_6009,12-Sep-2022,5900.0,8287.0
9,10,0900024,05-Jul-2023,T_9037,T_7650,29-Dec-2022,86109.0,96427.0


### Digital Loan Repayments Data

In [15]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('J_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('J_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from jumia.transactions_dimension td
    inner join jumia.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join jumia.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample TANDA.csv', index=False)

[2023-07-05 10:11:08,821] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,05-Jul-2023,J_946,J_167,4471.0,0.0,0,0.0,27-May-2021,2843.0
1,2,0900024,05-Jul-2023,J_946,J_167,2.0,0.0,0,0.0,02-Jun-2021,4469.0
2,3,0900024,05-Jul-2023,J_946,J_167,7314.0,0.0,0,0.0,20-May-2021,2686.0
3,4,0900024,05-Jul-2023,J_946,J_186,6505.0,0.0,0,0.0,30-Jun-2021,8495.0
4,5,0900024,05-Jul-2023,J_946,J_186,15000.0,0.0,0,0.0,23-Jun-2021,7500.0
...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,J_6231,J_4244,1.0,0.0,0,0.0,17-Mar-2022,3999.0
96,97,0900024,05-Jul-2023,J_5999,J_4505,1.0,0.0,0,0.0,21-Mar-2022,4099.0
97,98,0900024,05-Jul-2023,J_6079,J_3996,3.0,0.0,0,0.0,09-Mar-2022,8297.0
98,99,0900024,05-Jul-2023,J_5625,J_3998,3.0,0.0,0,0.0,01-Mar-2022,9097.0


### Customer Data

In [16]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('J_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from jumia.client_summary_view cmsv
        inner join jumia.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-07-05 10:12:50,180] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,05-Jul-2023,J_10035,CUSTID01,31126787,None,KE,M,James,Maina,Muchiri,None,27-Oct-2022,254706860752
1,2,0900024,05-Jul-2023,J_10081,CUSTID01,9705195,None,KE,M,Kennedy,None,Murithi,None,04-Nov-2022,254701423528
2,3,0900024,05-Jul-2023,J_10082,CUSTID01,34284368,None,KE,M,kelvin,None,Mawira,None,04-Nov-2022,254796019882
3,4,0900024,05-Jul-2023,J_10100,CUSTID01,27744800,None,KE,M,Christine,None,Nkatha,None,11-Nov-2022,254710392472
4,5,0900024,05-Jul-2023,J_10212,CUSTID01,27193391,None,KE,M,Joseph,None,Mungai,None,16-Dec-2022,254743881359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,J_9602,CUSTID01,32789820,None,KE,M,Danson,Ngugi,Mwaura,None,22-Sep-2022,254705520500
96,97,0900024,05-Jul-2023,J_9792,CUSTID01,35644604,None,KE,M,Salah,None,Adan,None,06-Oct-2022,None
97,98,0900024,05-Jul-2023,J_9951,CUSTID01,34387832,None,KE,M,Robert,None,Wachira,None,19-Oct-2022,254723957250
98,99,0900024,05-Jul-2023,J_9953,CUSTID01,32233665,None,KE,M,Cynthia,None,Pendo,None,19-Oct-2022,254701794014


# COPIA

### Digital Loans Account Data

In [18]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('C_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('C_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from copia.loans_fact_table_summary_view lftmsv
    inner join copia.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join copia.product_summary_view pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join copia.loan_status_view lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample TANDA.csv', index=False)

[2023-07-05 10:29:30,349] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,05-Jul-2023,J_2381,J_986,MEREZA AYALO,04-Oct-2022,9180.0,0.0,0,5949.0,0,0,0,184.0,CHN02,1,15313.0,CLOSED
1,2,0900024,05-Jul-2023,J_2589,J_1037,CONSOLATE OLUOCH,25-Nov-2022,9180.0,2662.0,0,5949.0,0,0,0,0.0,CHN02,1,15129.0,ACTIVE
2,3,0900024,05-Jul-2023,J_2374,J_982,JOSHUA OLOO,04-Oct-2022,13250.0,0.0,0,8586.0,0,0,0,51.0,CHN02,1,21887.0,CLOSED
3,4,0900024,05-Jul-2023,J_2486,J_1000,KELVIN MUTISYA,24-Oct-2022,9180.0,3616.0,0,5949.0,0,0,0,323.0,CHN02,1,15452.0,ACTIVE
4,5,0900024,05-Jul-2023,J_2393,J_991,DANIEL THEBERE,08-Oct-2022,13250.0,0.0,0,8586.0,0,0,0,0.0,CHN02,1,21836.0,CLOSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,J_2597,J_1044,SAMWEL OCHODHO,25-Nov-2022,13250.0,15968.0,0,8586.0,0,0,0,1532.0,CHN02,1,23368.0,ACTIVE
96,97,0900024,05-Jul-2023,J_2598,J_1045,MONICAH ATIENO,25-Nov-2022,9180.0,11078.0,0,5949.0,0,0,0,1076.0,CHN02,1,16205.0,ACTIVE
97,98,0900024,05-Jul-2023,J_2614,J_1057,JAMES KAMAU,29-Nov-2022,9180.0,11144.0,0,5949.0,0,0,0,1214.0,CHN02,1,16343.0,ACTIVE
98,99,0900024,05-Jul-2023,J_2528,J_1021,JANEFFER STEPHEN,08-Nov-2022,9180.0,3362.0,0,5949.0,0,0,0,60.0,CHN02,1,15189.0,ACTIVE


### Overdue (Non-Performing) Digital Loans

In [20]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('C_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('C_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from copia.loans_fact_table_summary_view lftmsv
    inner join copia.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample TANDA.csv', index=False)

[2023-07-05 10:47:36,815] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT


### Digital Loan Repayments Data

In [23]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('C_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('C_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from copia.transactions_dimension_view td
    inner join copia.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join copia.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample TANDA.csv', index=False)

[2023-07-05 10:51:44,324] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,05-Jul-2023,C_2381,C_986,2040.0,0.0,0,0.0,15-May-2023,1020.0
1,2,0900024,05-Jul-2023,C_2381,C_986,4097.0,0.0,0,0.0,04-Mar-2023,1003.0
2,3,0900024,05-Jul-2023,C_2381,C_986,5100.0,0.0,0,0.0,08-Feb-2023,1020.0
3,4,0900024,05-Jul-2023,C_2381,C_986,7140.0,0.0,0,0.0,05-Dec-2022,1020.0
4,5,0900024,05-Jul-2023,C_2381,C_986,8160.0,0.0,0,0.0,03-Nov-2022,1020.0
...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,C_2589,C_1037,7140.0,661.0,0,0.0,14-Jan-2023,0.0
96,97,0900024,05-Jul-2023,C_2589,C_1037,7140.0,661.0,0,0.0,13-Jan-2023,0.0
97,98,0900024,05-Jul-2023,C_2589,C_1037,7140.0,661.0,0,0.0,11-Jan-2023,0.0
98,99,0900024,05-Jul-2023,C_2589,C_1037,7140.0,661.0,0,0.0,09-Jan-2023,6.0


### Customer Data

In [25]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('C_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from copia.client_summary_view cmsv
        inner join copia.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-07-05 10:52:57,053] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,05-Jul-2023,C_2686,CUSTID01,22684386,None,KE,None,SHADRACK,None,KARANI,01-Jan-1982,12-Dec-2022,254720002906
1,2,0900024,05-Jul-2023,C_2685,CUSTID01,24554363,None,KE,None,MILKA,None,ONYANGO,01-Jan-1984,12-Dec-2022,254710537658
2,3,0900024,05-Jul-2023,C_2682,CUSTID01,10422013,None,KE,None,HALIMA,None,LUBANGA,24-Dec-1970,12-Dec-2022,254712305071
3,4,0900024,05-Jul-2023,C_2675,CUSTID01,23748210,None,KE,None,BEATRICE,None,CHEMUTAI,03-Jul-1984,10-Dec-2022,254726766394
4,5,0900024,05-Jul-2023,C_2674,CUSTID01,23678588,None,KE,None,FRANCIS,None,KAMAU,29-Jan-1984,10-Dec-2022,254724578708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,C_2343,CUSTID01,26378296,None,KE,None,RUTH,None,BIU,24-Aug-1988,23-Sep-2022,254728654902
96,97,0900024,05-Jul-2023,C_2338,CUSTID01,28525017,None,KE,None,TERESIA,None,WANYOIKE,10-Oct-1991,21-Sep-2022,254720982213
97,98,0900024,05-Jul-2023,C_2337,CUSTID01,13391516,None,KE,None,MARY,None,MURITU,28-Aug-1974,20-Sep-2022,254720369615
98,99,0900024,05-Jul-2023,C_2335,CUSTID01,32603762,None,KE,None,GRACE,None,KARANJA,23-Dec-1994,20-Sep-2022,254746210752


# JUBILEE

### Digital Loans Account Data

In [27]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('J_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('J_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from jubilee.loans_fact_table_summary_view lftmsv
    inner join jubilee.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join jubilee.product_dimension pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join jubilee.loan_status_dimension lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample TANDA.csv', index=False)

[2023-07-05 10:55:48,612] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,05-Jul-2023,J_369,J_1102,ROSE OMARIBA,13-Dec-2022,46143.0,28901.0,0,7348.0,0,0,0,0.0,CHN02,30,53491.0,ACTIVE
1,2,0900024,05-Jul-2023,J_257,J_1134,EDWARD NDUATI,01-Feb-2023,245259.0,189528.0,0,39033.0,0,0,0,0.0,CHN02,30,284292.0,ACTIVE
2,3,0900024,05-Jul-2023,J_399,J_1133,LUCY WAFULA,01-Feb-2023,57166.0,36791.0,0,7213.0,0,0,0,0.0,CHN02,30,64379.0,ACTIVE
3,4,0900024,05-Jul-2023,J_383,J_1126,MARGARET RUTERE,29-Dec-2022,80476.0,41428.0,0,12812.0,0,0,0,0.0,CHN02,30,93288.0,ACTIVE
4,5,0900024,05-Jul-2023,J_234,J_1014,BONFACE GATOBU,29-Jul-2022,149204.0,48147.0,0,19667.0,0,0,0,0.0,CHN02,30,168871.0,ACTIVE
5,6,0900024,05-Jul-2023,J_257,J_1131,EDWARD NDUATI,04-Jan-2023,87526.0,56368.0,0,13934.0,0,0,0,0.0,CHN02,30,101460.0,ACTIVE
6,7,0900024,05-Jul-2023,J_371,J_1135,JAMES MASAI,07-Feb-2023,63816.0,57538.0,0,10160.0,0,0,0,0.0,CHN02,30,73976.0,ACTIVE
7,8,0900024,05-Jul-2023,J_115,J_1122,WILLIAM IRUNGU,16-Dec-2022,59363.0,61170.0,0,9453.0,0,0,0,0.0,CHN02,30,68816.0,ACTIVE
8,9,0900024,05-Jul-2023,J_302,J_979,JUDY NTHENYA COMPANY,06-May-2022,65647.0,7295.0,0,8656.0,0,0,0,0.0,CHN02,30,74303.0,ACTIVE
9,10,0900024,05-Jul-2023,J_377,J_1123,PERIS KIRUGA,19-Dec-2022,33932.0,36031.0,0,2099.0,0,0,0,0.0,CHN02,30,36031.0,ACTIVE


### Overdue (Non-Performing) Digital Loans

In [29]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('C_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('C_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from jubilee.loans_fact_table_summary_view lftmsv
    inner join jubilee.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample TANDA.csv', index=False)

[2023-07-05 10:56:59,599] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT
0,1,0900024,05-Jul-2023,C_316,C_986,19-Dec-2022,48622.0,51123.0
1,2,0900024,05-Jul-2023,C_199,C_1017,17-Jun-2023,37488.0,40532.0
2,3,0900024,05-Jul-2023,C_330,C_1015,30-Mar-2023,46073.0,48943.0
3,4,0900024,05-Jul-2023,C_377,C_1123,17-Jun-2023,33932.0,36031.0
4,5,0900024,05-Jul-2023,C_302,C_979,01-May-2023,7295.0,7295.0


### Digital Loan Repayments Data

In [32]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('J_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('J_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from jubilee.transactions_dimension td
    inner join jubilee.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join jubilee.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample TANDA.csv', index=False)

[2023-07-05 12:10:31,261] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,05-Jul-2023,C_33,C_2,26687.0,0.0,0,0.0,04-Jan-2021,8455.0
1,2,0900024,05-Jul-2023,C_33,C_2,9124.0,0.0,0,0.0,05-Mar-2021,8892.0
2,3,0900024,05-Jul-2023,C_33,C_2,18016.0,0.0,0,0.0,03-Feb-2021,8671.0
3,4,0900024,05-Jul-2023,C_33,C_2,0.0,0.0,0,0.0,06-Apr-2021,9124.0
4,5,0900024,05-Jul-2023,C_46,C_3,0.0,0.0,0,0.0,30-Jan-2021,14620.0
...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,C_146,C_980,0.0,0.0,0,0.0,28-Sep-2022,10269.0
96,97,0900024,05-Jul-2023,C_146,C_980,10269.0,0.0,0,0.0,01-Sep-2022,10013.0
97,98,0900024,05-Jul-2023,C_146,C_980,20282.0,0.0,0,0.0,02-Aug-2022,9764.0
98,99,0900024,05-Jul-2023,C_146,C_980,30046.0,0.0,0,0.0,02-Jul-2022,9522.0


### Customer Data

In [34]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('J_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from jubilee.client_summary_view cmsv
        inner join jubilee.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-07-05 12:11:13,419] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,05-Jul-2023,J_300,CUSTID02,PT8535766,None,KE,None,VIOLET,None,QUAIL,None,14-Apr-2022,254723522131
1,2,0900024,05-Jul-2023,J_33,CUSTID02,C026065,None,KE,None,CATHERINE,None,WAWERU,None,11-Jul-2020,254722519526
2,3,0900024,05-Jul-2023,J_330,CUSTID02,N/A,None,KE,None,JEYJEY,AND,COMPANY LIMITED A/A,None,02-Aug-2022,254724542602
3,4,0900024,05-Jul-2023,J_72,CUSTID02,A003173667R,None,KE,None,CAROLINE,None,WANJIKU,None,09-Sep-2020,254729840520
4,5,0900024,05-Jul-2023,J_107,CUSTID01,25629107,None,KE,None,PAUL,None,NDUNGU,None,09-Mar-2021,254722923009
5,6,0900024,05-Jul-2023,J_115,CUSTID01,27455988,None,KE,None,WILLIAM,None,IRUNGU,None,05-Apr-2021,254702878743
6,7,0900024,05-Jul-2023,J_139,CUSTID01,22009321,None,KE,None,DANIEL,None,MWANGI,None,25-May-2021,254722141401
7,8,0900024,05-Jul-2023,J_143,CUSTID01,24726334,None,KE,None,DOREEN,None,MBAE,None,05-Jun-2021,254726150076
8,9,0900024,05-Jul-2023,J_146,CUSTID01,26578880,None,KE,None,VINCENT,None,ODHIAMBO,None,09-Jun-2021,254729377006
9,10,0900024,05-Jul-2023,J_150,CUSTID01,14569256,None,KE,None,JAMES,None,MUTURI,None,11-Jun-2021,254721913852


# KENYA AIRWAYS

### Digital Loans Account Data

In [38]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('KQ_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('KQ_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from kenya_airways.loans_fact_table_summary_view lftmsv
    inner join kenya_airways.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join kenya_airways.product_summary_view pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join kenya_airways.loan_status_view lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample TANDA.csv', index=False)

[2023-07-05 12:24:14,979] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,05-Jul-2023,KQ_1437,KQ_560,ZAWADI BUSHOKI,28-Apr-2022,100.0,0.0,0,3.0,0,0,0,0.0,CHN02,14,103.0,CLOSED
1,2,0900024,05-Jul-2023,KQ_1679,KQ_661,FIONA KAMAU,06-Jun-2022,100.0,0.0,0,3.0,0,0,0,0.0,CHN02,14,103.0,CLOSED
2,3,0900024,05-Jul-2023,KQ_1894,KQ_769,KALUTU FRANCIS,13-Jul-2022,100.0,0.0,0,3.0,0,0,0,3.0,CHN02,14,106.0,CLOSED
3,4,0900024,05-Jul-2023,KQ_640,KQ_502,CAROLINE KAVITA,22-Mar-2022,100.0,0.0,0,4.0,0,0,0,0.0,CHN02,14,104.0,CLOSED
4,5,0900024,05-Jul-2023,KQ_1106,KQ_511,MARTIN NJOROGE,23-Mar-2022,100.0,0.0,0,3.0,0,0,0,10.0,CHN02,14,113.0,CLOSED
5,6,0900024,05-Jul-2023,KQ_1424,KQ_556,DENNIS BUNDI,26-Apr-2022,100.0,0.0,0,3.0,0,0,0,3.0,CHN02,14,106.0,CLOSED
6,7,0900024,05-Jul-2023,KQ_1437,KQ_563,ZAWADI BUSHOKI,28-Apr-2022,100.0,0.0,0,3.0,0,0,0,3.0,CHN02,14,106.0,CLOSED
7,8,0900024,05-Jul-2023,KQ_1438,KQ_566,MERCY MAINA,28-Apr-2022,100.0,0.0,0,3.0,0,0,0,2.0,CHN02,14,105.0,CLOSED
8,9,0900024,05-Jul-2023,KQ_1909,KQ_777,CAROLINE MIBEI,14-Jul-2022,100.0,0.0,0,3.0,0,0,0,3.0,CHN02,14,106.0,CLOSED


### Overdue (Non-Performing) Digital Loans

In [40]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('C_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('C_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from kenya_airways.loans_fact_table_summary_view lftmsv
    inner join kenya_airways.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample TANDA.csv', index=False)

[2023-07-05 12:24:53,326] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT


### Digital Loan Repayments Data

In [44]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('KQ_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('KQ_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from kenya_airways.transactions_dimension_view td
    inner join kenya_airways.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join kenya_airways.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample TANDA.csv', index=False)

[2023-07-05 12:28:04,507] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID
0,1,0900024,05-Jul-2023,KQ_1437,KQ_560,0.0,0.0,0,0.0,28-Apr-2022,100.0
1,2,0900024,05-Jul-2023,KQ_1679,KQ_661,0.0,0.0,0,0.0,06-Jun-2022,100.0
2,3,0900024,05-Jul-2023,KQ_1894,KQ_769,0.0,0.0,0,0.0,28-Jul-2022,100.0
3,4,0900024,05-Jul-2023,KQ_640,KQ_502,0.0,0.0,0,0.0,05-Apr-2022,100.0
4,5,0900024,05-Jul-2023,KQ_640,KQ_502,100.0,0.0,0,0.0,22-Mar-2022,0.0
5,6,0900024,05-Jul-2023,KQ_1106,KQ_511,0.0,0.0,0,0.0,06-Apr-2022,100.0
6,7,0900024,05-Jul-2023,KQ_1424,KQ_556,0.0,0.0,0,0.0,10-May-2022,100.0
7,8,0900024,05-Jul-2023,KQ_1437,KQ_563,0.0,0.0,0,0.0,12-May-2022,100.0
8,9,0900024,05-Jul-2023,KQ_1438,KQ_566,16.0,0.0,0,0.0,25-May-2022,2.0
9,10,0900024,05-Jul-2023,KQ_1438,KQ_566,18.0,0.0,0,0.0,25-May-2022,10.0


### Customer Data

In [45]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('J_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from kenya_airways.client_summary_view cmsv
        inner join kenya_airways.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-07-05 12:29:08,913] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,05-Jul-2023,J_1106,CUSTID01,23937032,None,KE,None,MARTIN,None,NJOROGE,10-Jan-1984,23-Mar-2022,254720807716
1,2,0900024,05-Jul-2023,J_1424,CUSTID01,29933266,None,KE,None,DENNIS,None,BUNDI,27-Nov-1993,26-Apr-2022,254715367066
2,3,0900024,05-Jul-2023,J_1437,CUSTID01,36073675,None,KE,None,ZAWADI,None,BUSHOKI,08-Apr-1992,27-Apr-2022,254707513122
3,4,0900024,05-Jul-2023,J_1438,CUSTID01,27414260,None,KE,None,MERCY,None,MAINA,27-Apr-2022,27-Apr-2022,254723579395
4,5,0900024,05-Jul-2023,J_1679,CUSTID01,35706057,None,KE,None,FIONA,None,KAMAU,27-Jun-1998,03-Jun-2022,254741863223
5,6,0900024,05-Jul-2023,J_1894,CUSTID01,13094244,None,KE,None,KALUTU,None,FRANCIS,13-Apr-2022,13-Jul-2022,254701674030
6,7,0900024,05-Jul-2023,J_1909,CUSTID01,32883455,None,KE,None,CAROLINE,None,MIBEI,14-Jul-2022,14-Jul-2022,254720632728
7,8,0900024,05-Jul-2023,J_1969,CUSTID01,24003271,None,KE,None,FRIDA,None,OWEGI,24-Mar-1985,26-Jul-2022,254723365660
8,9,0900024,05-Jul-2023,J_640,CUSTID01,27353072,None,KE,None,CAROLINE,None,KAVITA,01-Jan-1980,27-Oct-2021,254726662669


# TWIGA

### Digital Loans Account Data

In [65]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loans Account Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('T_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('T_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        concat(cmsv.first_name, ' ', cmsv.middle_name, ' ', cmsv.last_name) as "ACCOUNT_NAME",
        to_char(disbursed_on_date, 'DD-Mon-YYYY') as "DISBURSEMENT_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_disbursed, 0) as "PRINCIPAL_LOAN_AMOUNT_DISBURSED",
        coalesce(total_outstanding, 0) as "OUTSTANDING_LOAN_BALANCE",
        0 as "DEBIT_INTEREST_RATE",
        coalesce(interest_charged, 0) as "FACILITY_ARRANGEMENT_FEE_CHARGE",
        0 as "UPFRONT_INTEREST_FEES_CHARGE",
        0 as "COST_OF_WALLET_TRANSFER", --except for Solv
        0 as "INSURANCE_FEE_CHARGE",
        coalesce(fee_charges_charged + penalty_charges_charged, 0) as "OTHER_CHARGE",
        dct.code as "CHANNEL_TYPE_USED",
        pd.repay_every as "DIGITAL_LOAN_TENOR",
        coalesce(total_expected_repayment, 0) as "INSTALMENT_AMOUNT",
        ast.code as "ACCOUNT_STATUS"
    from twiga.loans_fact_table_summary_view lftmsv
    inner join twiga.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    inner join twiga.product_dimension pd on lftmsv.product_surrogate_id = pd.surrogate_id
    inner join twiga.loan_status_dimension lsd on lftmsv.loan_status_surrogate_id = lsd.surrogate_id
    inner join central_bank_of_kenya.digital_channel_types dct on dct."type" = 'USSD'
    inner join central_bank_of_kenya.account_status_types ast on ast."type" = lsd.message
    where loan_status in (300, 600, 700)
    limit 100
    """
)#.to_csv('~/data/Digital Loans Account Data Sample TANDA.csv', index=False)

[2023-07-06 10:16:41,017] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,ACCOUNT_NAME,DISBURSEMENT_DATE,PRINCIPAL_LOAN_AMOUNT_DISBURSED,OUTSTANDING_LOAN_BALANCE,DEBIT_INTEREST_RATE,FACILITY_ARRANGEMENT_FEE_CHARGE,UPFRONT_INTEREST_FEES_CHARGE,COST_OF_WALLET_TRANSFER,INSURANCE_FEE_CHARGE,OTHER_CHARGE,CHANNEL_TYPE_USED,DIGITAL_LOAN_TENOR,INSTALMENT_AMOUNT,ACCOUNT_STATUS
0,1,0900024,06-Jul-2023,T_2201,T_2235,FAITH NDANU,08-Nov-2021,2470.0,0.0,0,22.0,0,0,0,23.0,CHN02,7,2515.0,CLOSED
1,2,0900024,06-Jul-2023,T_1784,T_2025,PAUL THUO NDUNGU,29-Oct-2021,1400.0,0.0,0,13.0,0,0,0,81.0,CHN02,7,1494.0,CLOSED
2,3,0900024,06-Jul-2023,T_2111,T_1843,SERAH NJERI WAWERU,22-Oct-2021,1700.0,0.0,0,16.0,0,0,0,54.0,CHN02,7,1770.0,CLOSED
3,4,0900024,06-Jul-2023,T_2784,T_1826,EUNICE AKINYI OLOO,21-Oct-2021,1100.0,0.0,0,10.0,0,0,0,40.0,CHN02,7,1150.0,CLOSED
4,5,0900024,06-Jul-2023,T_1817,T_2178,DANIEL MWANGI MAINA,05-Nov-2021,2000.0,0.0,0,19.0,0,0,0,157.0,CHN02,7,2176.0,CLOSED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,06-Jul-2023,T_1705,T_593,JANET NJERI NJOROGE,13-Aug-2021,1800.0,0.0,0,17.0,0,0,0,320.0,CHN02,7,2137.0,CLOSED
96,97,0900024,06-Jul-2023,T_2179,T_594,JOSEPH GICHOYA MUKURURO,13-Aug-2021,2900.0,0.0,0,27.0,0,0,0,54.0,CHN02,7,2981.0,CLOSED
97,98,0900024,06-Jul-2023,T_2874,T_597,PAULINE NUNGA NGIGI,13-Aug-2021,2200.0,0.0,0,20.0,0,0,0,62.0,CHN02,7,2282.0,CLOSED
98,99,0900024,06-Jul-2023,T_2897,T_598,JULIET WANJIRU IRUNGU,13-Aug-2021,1540.0,0.0,0,14.0,0,0,0,106.0,CHN02,7,1660.0,CLOSED


### Overdue (Non-Performing) Digital Loans

In [60]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Overdue (Non-Performing) Digital Loans
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('C_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('C_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        to_char(expected_matured_on_date + '90 day'::interval, 'DD-Mon-YYYY') as "OVERDUE_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        coalesce(principal_outstanding, 0) as "PRINCIPAL_ARREARS_AMOUNT",
        coalesce(total_outstanding, 0) as "TOTAL_OUTSTANDING_AMOUNT"
    from twiga.loans_fact_table_summary_view lftmsv
    inner join twiga.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where current_date - (expected_matured_on_date + '90 day'::interval)::date >= 1 and loan_status = 300
    limit 100
    """
)##.to_csv('~/data/Overdue (Non-Performing) Digital Loans Sample TANDA.csv', index=False)

[2023-07-05 14:27:33,550] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OVERDUE_DATE,PRINCIPAL_ARREARS_AMOUNT,TOTAL_OUTSTANDING_AMOUNT


### Digital Loan Repayments Data

In [62]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Digital Loan Repayments Data
    select
        row_number() over () as "ROW_ID",
        '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
        to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
        concat('KQ_', cmsv.mifos_id) as "CUSTOMER_ID",
        concat('KQ_', lftmsv.loan_mifos_id) as "LOAN_ACCOUNT_CONTRACT_NUMBER",
        coalesce(td.outstanding_loan_balance_derived, 0) as "OUTSTANDING_LOAN_BALANCE", -- data quality issue
        coalesce(lftmsv.interest_outstanding, 0) as "ACCRUED_INTEREST_BALANCE",  -- data quality issue
        0 as "INTEREST_CHARGED_ON_REPAYMENT",
        coalesce(td.fee_charges_portion_derived + td.penalty_charges_portion_derived, 0) as "LATE_PAYMENT_FEE_CHARGE",
        to_char(td.transaction_date, 'DD-Mon-YYYY') as "LOAN_REPAYMENT_DATE",
        coalesce(td.principal_portion_derived, 0) as "PRINCIPAL_LOAN_AMOUNT_REPAID"
    from twiga.transactions_dimension td
    inner join twiga.loans_fact_table_summary_view lftmsv on lftmsv.loan_mifos_id::varchar = td.mifos_loan_id
    inner join twiga.client_summary_view cmsv on lftmsv.client_surrogate_id = cmsv.surrogate_id
    where td.transaction_type_enum = 2 and lower(td.receipt_number) not like '%waiv%'
    limit 100
    """
)#.to_csv('~/data/Digital Loan Repayments Data Sample TANDA.csv', index=False)

[2023-07-05 14:31:32,796] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,LOAN_ACCOUNT_CONTRACT_NUMBER,OUTSTANDING_LOAN_BALANCE,ACCRUED_INTEREST_BALANCE,INTEREST_CHARGED_ON_REPAYMENT,LATE_PAYMENT_FEE_CHARGE,LOAN_REPAYMENT_DATE,PRINCIPAL_LOAN_AMOUNT_REPAID


### Customer Data

In [64]:
warehouse_hook.get_pandas_df(
    sql="""
    -- Customer Data
    with cust_dt as (
        select
            distinct
            '0900024' as "DIGITAL_CREDIT_PROVIDER_CODE",
            to_char(current_date, 'DD-Mon-YYYY') as "REPORTING_DATE", -- cannot be in format 'DD-Mon-YYYY' and still be dtype date
            concat('T_', cmsv.mifos_id) as "CUSTOMER_ID", -- store number vs national_id
            CASE WHEN cmsv.national_id ~ '[a-zA-Z]' THEN 'Passport' ELSE 'National Identity Card' END AS "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE", -- how to distinguish huduma number from national ID number
            cmsv.national_id as "NATIONAL_IDENTITY_PASSPORT_NUMBER", -- column name in template has slash
            null as "PERSONAL_IDENTIFICATION_NUMBER", --kra pin
            cmsv.gender,
            cmsv.first_name as "FIRST_NAME",
            cmsv.middle_name as "MIDDLE_NAME",
            cmsv.last_name as "SURNAME",
            to_char(cmsv.date_of_birth, 'DD-Mon-YYYY') as "DATE_OF_BIRTH_REGISTRATION", -- cannot be in format 'DD-MM-YYYY' and still be dtype date
            to_char(cmsv.submitted_on_date, 'DD-Mon-YYYY') as "ONBOARDING_DATE",
            cmsv.mobile_number as "MOBILE_NUMBER"
        from twiga.client_summary_view cmsv
        inner join twiga.loans_fact_table_summary_view lftmsv on cmsv.surrogate_id = lftmsv.client_surrogate_id
    ) select
        row_number() over () as "ROW_ID", "DIGITAL_CREDIT_PROVIDER_CODE", "REPORTING_DATE", "CUSTOMER_ID", cpdt.code as "PRIMARY_IDENTIFICATION_DOCUMENT_TYPE",
        "NATIONAL_IDENTITY_PASSPORT_NUMBER", "PERSONAL_IDENTIFICATION_NUMBER", icc.country_code as "NATIONALITY", gt.code as "GENDER", "FIRST_NAME",
        case when "SURNAME" is null and "MIDDLE_NAME" is not null then null else "MIDDLE_NAME" end as "MIDDLE_NAME",
        case when "SURNAME" is null then "MIDDLE_NAME" else "SURNAME" end as "SURNAME",
        "DATE_OF_BIRTH_REGISTRATION", "ONBOARDING_DATE", "MOBILE_NUMBER"
    from cust_dt
    left join central_bank_of_kenya.cust_primary_doc_types cpdt on cpdt."type" = cust_dt."PRIMARY_IDENTIFICATION_DOCUMENT_TYPE"
    left join central_bank_of_kenya.gender_types gt on gt."type" = cust_dt.gender
    left join central_bank_of_kenya.iso_country_codes icc on icc.country_description = 'Kenya'
    limit 100
    """
)##.to_csv(f'~/data/CBK Customer Data Sample SOLV {datetime.datetime.now()}.csv', index=False)

[2023-07-05 14:34:37,349] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.


/home/henrykuria/airflow_venv/lib/python3.10/site-packages/pandas/io/sql.py:761 UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

,ROW_ID,DIGITAL_CREDIT_PROVIDER_CODE,REPORTING_DATE,CUSTOMER_ID,PRIMARY_IDENTIFICATION_DOCUMENT_TYPE,NATIONAL_IDENTITY_PASSPORT_NUMBER,PERSONAL_IDENTIFICATION_NUMBER,NATIONALITY,GENDER,FIRST_NAME,MIDDLE_NAME,SURNAME,DATE_OF_BIRTH_REGISTRATION,ONBOARDING_DATE,MOBILE_NUMBER
0,1,0900024,05-Jul-2023,T_1546,CUSTID01,22468538,None,KE,None,ELIZABETH,NJERI,WAMBUI,06-Jun-1986,12-Jul-2021,+254723959262
1,2,0900024,05-Jul-2023,T_1553,CUSTID01,22276954,None,KE,None,CAROLINE,MUHINYE,KIMANI,06-Jun-1986,12-Jul-2021,+254758853339
2,3,0900024,05-Jul-2023,T_1559,CUSTID01,1892759,None,KE,None,ROSE,None,MUTHEO,06-Jun-1986,12-Jul-2021,+254721303217
3,4,0900024,05-Jul-2023,T_1583,CUSTID01,25218514,None,KE,None,NAZNEEN,MEHBOOB,MOHAMMED SALEH,06-Jun-1986,12-Jul-2021,+254722724798
4,5,0900024,05-Jul-2023,T_1594,CUSTID01,30076705,None,KE,None,SAMUEL,KIOKO,DAVID,06-Jun-1986,12-Jul-2021,+254708046057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,0900024,05-Jul-2023,T_2758,CUSTID01,7235027,None,KE,None,SARAH,KATHULE,MUSILA,06-Jun-1986,07-Aug-2021,+254713479042
96,97,0900024,05-Jul-2023,T_2773,CUSTID01,11129272,None,KE,None,JANE,WANGARI,KIGOTHO,06-Jun-1986,09-Aug-2021,+254722466393
97,98,0900024,05-Jul-2023,T_2775,CUSTID01,21971602,None,KE,None,MARTHA,WANJIRU,MUTURI,06-Jun-1986,09-Aug-2021,+254700829987
98,99,0900024,05-Jul-2023,T_2784,CUSTID01,23510022,None,KE,None,EUNICE,AKINYI,OLOO,06-Jun-1986,09-Aug-2021,+254724954862


In [8]:
pd.read_csv('~/data/CBK Customer Data Sample Bloom 2.0 2023-06-27.csv')['las'].notna()

KeyError: 'surname'

#### Metadata

In [17]:
path = '~/data/DCP Standard Data.xlsx'

In [18]:
dt = pd.ExcelFile(path)

In [20]:
#### DCPs - GL Codes
gls = pd.read_excel(path, sheet_name='DCPs - GL Codes', skiprows=2).rename(columns={
    'GL CODE': 'code', 'GL DESCRIPTION': 'description', 'GL TYPE': 'type'
})[['code', 'description', 'type']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.general_ledgers',
    target_fields=gls.reindex().columns.tolist(),
    replace=False,
    rows=tuple(gls.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:17:56,462] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:18:07,460] {sql.py:462} INFO - Done loading. Loaded a total of 40 rows into central_bank_of_kenya.general_ledgers


In [24]:
#### Classification of complaints
ccc = pd.read_excel(path, sheet_name='Classification of complaints', skiprows=2)
ccc.rename(columns={
    'Customer Complaint Classification': 'classification',
    'Customer Complaint Classification Type': 'type',
    'Customer Complaint Calssification Type Details': 'details'
}, inplace=True)
ccc = ccc[['classification', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.complaints',
    target_fields=ccc.reindex().columns.tolist(),
    replace=False,
    rows=tuple(ccc.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:20:59,216] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:21:02,002] {sql.py:462} INFO - Done loading. Loaded a total of 2 rows into central_bank_of_kenya.complaints


In [28]:
#### County & Sub Counties
csc = pd.read_excel(path, sheet_name='County & Sub Counties', skiprows=2)
csc.rename(columns={
    'SUB-COUNTY CODE': 'sub_county_code',
    'SUB-COUNTY NAME': 'sub_county_name',
    'COUNTY CODE': 'county_code',
    'COUNTY NAME': 'county_name',
}, inplace=True)
csc = csc[['sub_county_code', 'sub_county_name', 'county_code', 'county_name']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.counties_sub_counties',
    target_fields=csc.reindex().columns.tolist(),
    replace=False,
    rows=tuple(csc.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:24:08,856] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:24:31,554] {sql.py:459} INFO - Loaded 100 rows into central_bank_of_kenya.counties_sub_counties so far
[2023-06-06 06:24:52,548] {sql.py:459} INFO - Loaded 200 rows into central_bank_of_kenya.counties_sub_counties so far
[2023-06-06 06:25:13,974] {sql.py:459} INFO - Loaded 300 rows into central_bank_of_kenya.counties_sub_counties so far
[2023-06-06 06:25:25,630] {sql.py:462} INFO - Done loading. Loaded a total of 349 rows into central_bank_of_kenya.counties_sub_counties


In [30]:
#### Nature of Complaint
noc = pd.read_excel(path, sheet_name='Nature of Complaint', skiprows=2)
noc.rename(columns={
    'Nature of Complaint': 'nature',
    'Nature of Complaint Type': 'type',
    'Nature of Complaint Type Details': 'details'
}, inplace=True)
noc = noc[['nature', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.nature_of_complaints',
    target_fields=noc.reindex().columns.tolist(),
    replace=False,
    rows=tuple(noc.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:26:59,555] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:27:03,630] {sql.py:462} INFO - Done loading. Loaded a total of 9 rows into central_bank_of_kenya.nature_of_complaints


In [32]:
#### Cust. Pri. ID. Doc. Types
cpidt = pd.read_excel(path, sheet_name='Cust. Pri. ID. Doc. Types', skiprows=2)
cpidt.rename(columns={
    'Document Code': 'code',
    'ID Document Type': 'type',
    'Document Type Description': 'description'
}, inplace=True)
cpidt = cpidt[['code', 'type', 'description']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.cust_primary_doc_types',
    target_fields=cpidt.reindex().columns.tolist(),
    replace=False,
    rows=tuple(cpidt.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:28:56,099] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:28:59,460] {sql.py:462} INFO - Done loading. Loaded a total of 6 rows into central_bank_of_kenya.cust_primary_doc_types


In [34]:
#### Gender Types
gt = pd.read_excel(path, sheet_name='Gender Types', skiprows=2)
gt.rename(columns={
    'Gender_Code': 'code',
    'Gender_Type': 'type',
    'Gender_Description': 'description'
}, inplace=True)
gt = gt[['code', 'type', 'description']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.gender_types',
    target_fields=gt.reindex().columns.tolist(),
    replace=False,
    rows=tuple(gt.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:31:46,020] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:31:49,280] {sql.py:462} INFO - Done loading. Loaded a total of 4 rows into central_bank_of_kenya.gender_types


In [36]:
#### Occupation Types
ot = pd.read_excel(path, sheet_name='Occupation Types', skiprows=2)
ot.rename(columns={
    'Occupation Code': 'code',
    'Occupation Types': 'type',
    'Occupation Type Details': 'details'
}, inplace=True)
ot = ot[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.occupation_types',
    target_fields=ot.reindex().columns.tolist(),
    replace=False,
    rows=tuple(ot.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:33:50,926] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:33:54,237] {sql.py:462} INFO - Done loading. Loaded a total of 5 rows into central_bank_of_kenya.occupation_types


In [38]:
#### ISO Country Codes
icc = pd.read_excel(path, sheet_name='ISO Country Codes', skiprows=2)
icc.rename(columns={
    'Country Code': 'country_code',
    'Country Description': 'country_description',
    'Continent Name': 'continent_name',
    'Continent Code': 'continent_code',
    'Sub-Region': 'sub_region'
}, inplace=True)
icc = icc[['country_code', 'country_description', 'sub_region', 'continent_name', 'continent_code']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.iso_country_codes',
    target_fields=icc.reindex().columns.tolist(),
    replace=False,
    rows=tuple(icc.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 06:38:07,537] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 06:38:33,964] {sql.py:459} INFO - Loaded 100 rows into central_bank_of_kenya.iso_country_codes so far
[2023-06-06 06:38:57,647] {sql.py:459} INFO - Loaded 200 rows into central_bank_of_kenya.iso_country_codes so far
[2023-06-06 06:39:10,112] {sql.py:462} INFO - Done loading. Loaded a total of 249 rows into central_bank_of_kenya.iso_country_codes


In [40]:
#### Marital Status Types
mst = pd.read_excel(path, sheet_name='Marital Status Types', skiprows=2)
mst.rename(columns={
    'Marital Status Code': 'code',
    'Marital Status Type Details': 'details',
    'Marital Status Types': 'type'
}, inplace=True)
mst = mst[['code', 'details', 'type']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.marital_status_types',
    target_fields=mst.reindex().columns.tolist(),
    replace=False,
    rows=tuple(mst.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 08:50:22,764] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 08:50:25,817] {sql.py:462} INFO - Done loading. Loaded a total of 6 rows into central_bank_of_kenya.marital_status_types


In [44]:
#### Loan Application Status
las = pd.read_excel(path, sheet_name='Loan Application Status', skiprows=2)
las.rename(columns={
    'Loan Application Status Type Code': 'code',
    'Loan Application Status  Details': 'details',
    'Loan Application Status  Type': 'type'
}, inplace=True)
las = las[['code', 'details', 'type']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.loan_application_statuses',
    target_fields=las.reindex().columns.tolist(),
    replace=False,
    rows=tuple(las.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 08:53:16,726] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 08:53:19,211] {sql.py:462} INFO - Done loading. Loaded a total of 4 rows into central_bank_of_kenya.loan_application_statuses


In [46]:
#### ISIC Economic Activies
EA = pd.read_excel(path, sheet_name='ISIC Economic Activies', skiprows=2)
EA.rename(columns={
    'DIVISIONS': 'divisions',
    'BUSINESS ECONOMIC ACTIVITY CODE': 'code',
    'BUSINESS ECONOMIC ACTIVITY SECTION': 'section'
}, inplace=True)
EA = EA[['divisions', 'code', 'section']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.economic_activities',
    target_fields=EA.reindex().columns.tolist(),
    replace=False,
    rows=tuple(EA.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 08:55:08,597] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 08:55:14,101] {sql.py:462} INFO - Done loading. Loaded a total of 21 rows into central_bank_of_kenya.economic_activities


In [55]:
#### ISO Currency Codes
cc = pd.read_excel(path, sheet_name='ISO Currency Codes', skiprows=2)
cc['Minor unit'] = cc['Minor unit'].apply(
    lambda x: np.NAN if str(x) == 'N.A.' else x
)
cc.rename(columns={
    'Alphabetic Code': 'alphabetic_code',
    'Numeric Code': 'numeric_code',
    'Minor unit': 'minor_unit',
    'Currency': 'currency'
}, inplace=True)
cc = cc[['alphabetic_code', 'numeric_code', 'minor_unit', 'currency']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.currency_codes',
    target_fields=cc.reindex().columns.tolist(),
    replace=False,
    rows=tuple(cc.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:02:23,924] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:02:43,542] {sql.py:459} INFO - Loaded 100 rows into central_bank_of_kenya.currency_codes so far
[2023-06-06 09:02:57,976] {sql.py:462} INFO - Done loading. Loaded a total of 179 rows into central_bank_of_kenya.currency_codes


In [54]:
#### Management Category Types
mct = pd.read_excel(path, sheet_name='Management Category Types', skiprows=2)
mct.rename(columns={
    'Management Category Type Code': 'code',
    'Management Category Type': 'type'
}, inplace=True)
mct = mct[['code', 'type']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.management_category_types',
    target_fields=mct.reindex().columns.tolist(),
    replace=False,
    rows=tuple(mct.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:02:20,683] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:02:23,486] {sql.py:462} INFO - Done loading. Loaded a total of 3 rows into central_bank_of_kenya.management_category_types


In [65]:
#### Executive Category Types
ect = pd.read_excel(path, sheet_name='Executive Category Types', skiprows=2)
ect.rename(columns={
    'Director Category Type Code': 'code',
    'Director Category Type': 'type',
    'Director Category Type Details *': 'details'
}, inplace=True)
ect = ect[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.executive_category_types',
    target_fields=ect.reindex().columns.tolist(),
    replace=False,
    rows=tuple(ect.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:15:19,034] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:15:21,471] {sql.py:462} INFO - Done loading. Loaded a total of 4 rows into central_bank_of_kenya.executive_category_types


In [64]:
#### Academic Qualifications
aq = pd.read_excel(path, sheet_name='Academic Qualifications', skiprows=2)
aq.rename(columns={
    'Academic Qualifications Code': 'code',
    'Academic Qualifications Type': 'type',
    'Academic Qualifications Type Details': 'details'
}, inplace=True)
aq = aq[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.academic_qualifications',
    target_fields=aq.reindex().columns.tolist(),
    replace=False,
    rows=tuple(aq.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:15:15,190] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:15:18,484] {sql.py:462} INFO - Done loading. Loaded a total of 7 rows into central_bank_of_kenya.academic_qualifications


In [63]:
#### Professional Qualifications
pq = pd.read_excel(path, sheet_name='Professional Qualifications', skiprows=2)
pq.rename(columns={
    'Professonal Qualifications Code': 'code',
    'Professonal Qualifications Type': 'type',
    'Professonal Qualifications Type Details': 'details'
}, inplace=True)
pq = pq[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.professional_qualifications',
    target_fields=pq.reindex().columns.tolist(),
    replace=False,
    rows=tuple(pq.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:14:41,759] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:14:45,382] {sql.py:462} INFO - Done loading. Loaded a total of 9 rows into central_bank_of_kenya.professional_qualifications


In [68]:
#### Digital Channel Types
dct = pd.read_excel(path, sheet_name='Digital Channel Types', skiprows=2)
dct.rename(columns={
    'Channels Type Code': 'code',
    'Channels Type': 'type',
    'Channels Type Code Details': 'details'
}, inplace=True)
dct = dct[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.digital_channel_types',
    target_fields=dct.reindex().columns.tolist(),
    replace=False,
    rows=tuple(dct.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:17:32,685] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:17:36,104] {sql.py:462} INFO - Done loading. Loaded a total of 5 rows into central_bank_of_kenya.digital_channel_types


In [73]:
#### Shareholding Flag
sf = pd.read_excel(path, sheet_name='Shareholding Flag', skiprows=2)
sf.rename(columns={
    'Shareholding Flag Type Code': 'code',
    'Committee Type Description': 'description',
    'Shareholding Flag Type Details': 'details'
}, inplace=True)
sf = sf[['code', 'description', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.shareholding_flags',
    target_fields=sf.reindex().columns.tolist(),
    replace=False,
    rows=tuple(sf.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:20:36,291] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:20:38,807] {sql.py:462} INFO - Done loading. Loaded a total of 2 rows into central_bank_of_kenya.shareholding_flags


In [76]:
#### Shareholder Types
st = pd.read_excel(path, sheet_name='Shareholder Types', skiprows=2)
st.rename(columns={
    'Shareholder Type Code': 'code',
    'Shareholder Type': 'type'
}, inplace=True)
st = st[['code', 'type']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.shareholder_types',
    target_fields=st.reindex().columns.tolist(),
    replace=False,
    rows=tuple(st.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:22:13,169] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:22:15,659] {sql.py:462} INFO - Done loading. Loaded a total of 3 rows into central_bank_of_kenya.shareholder_types


In [78]:
#### Customer Complaint Status Types
ccst = pd.read_excel(path, sheet_name='Customer Complaint Status Types', skiprows=2)
ccst.rename(columns={
    'Customer Complaint Status Type Code': 'code',
    'Customer Complaint Status Type Details': 'details',
    'Customer Complaint Status Type': 'type'
}, inplace=True)
ccst = ccst[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.customer_complaint_status_types',
    target_fields=ccst.reindex().columns.tolist(),
    replace=False,
    rows=tuple(ccst.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:24:31,386] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:24:33,964] {sql.py:462} INFO - Done loading. Loaded a total of 4 rows into central_bank_of_kenya.customer_complaint_status_types


In [81]:
#### Account Status Types
ast = pd.read_excel(path, sheet_name='Account Status Types', skiprows=2)
ast.rename(columns={
    'Account Status Code': 'code',
    'Account Status Type Description': 'description',
    'Account Status Type': 'type'
}, inplace=True)
ast = ast[['code', 'type', 'description']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.account_status_types',
    target_fields=ast.reindex().columns.tolist(),
    replace=False,
    rows=tuple(ast.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:26:49,199] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:26:52,527] {sql.py:462} INFO - Done loading. Loaded a total of 2 rows into central_bank_of_kenya.account_status_types


In [84]:
#### Service Status Flag
sst = pd.read_excel(path, sheet_name='Service Status Flag', skiprows=2)
sst.rename(columns={
    'Service Status Type Code': 'code',
    'Service Status Type Details': 'details',
    'Service Status Type': 'type'
}, inplace=True)
sst = sst[['code', 'type', 'details']]
warehouse_hook.insert_rows(
    table='central_bank_of_kenya.service_status_flags',
    target_fields=sst.reindex().columns.tolist(),
    replace=False,
    rows=tuple(sst.reindex().replace({np.NAN: None}).itertuples(index=False)),
    commit_every=100
)

[2023-06-06 09:36:18,889] {base.py:73} INFO - Using connection ID 'rds_afsg_ds_prod_postgresql_dwh' for task execution.
[2023-06-06 09:36:22,117] {sql.py:462} INFO - Done loading. Loaded a total of 4 rows into central_bank_of_kenya.service_status_flags
